In [35]:
from resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing import image
from imagenet_utils import preprocess_input
import json
import cPickle as pk
import pandas as pd
import re
from time import time
import os
import numpy as np

base_model = ResNet50(weights='imagenet')

K.image_dim_ordering: th


In [23]:
def load_everything():
    # load configuration
    with open('../config.json', 'rb') as f:
        config = json.load(f)

    mainImPath = config['image_dir']
    pDir = config['pickle_dir']

    bb = pk.load(open(pDir + 'pDogs-bounding-boxes-clean.pd.pk', 'rb'))
    bb.dropna(inplace=True)
    bb.path = bb.path.apply(lambda x: re.sub('scrape-ims', 'scrape_ims', x))

    breeds = list(sorted(bb.breed.unique().tolist()))

    return breeds, bb, mainImPath

def test(timing=True, verbose=True):
    start = time()
    testImDir = mainImPath + breeds[0] + '/'
    testIms = os.listdir(testImDir)
    if verbose:
        print 'extracting features for', breeds[0]
        print 'file:', testIms[0]
    b4 = extract_feats(testImDir + testIms[0])
    if timing:
        runtime = time() - start
        if verbose:
            print 'took', runtime, 'seconds'

    return b4, runtime, testImDir + testIms[0]

def create_model(output_layer='fc1000'):
    model = Model(input=base_model.input, output=base_model.get_layer(output_layer).output)
    return model

def extract_feats(img_path):
        try:
            img = image.load_img(img_path, target_size=(224, 224))
        except IOError:
            print 'couldn\'t load file'
            return None

        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        features = model.predict(x)
        return features

In [25]:
breeds, bb, mainImPath = load_everything()
model = create_model()

In [26]:
len(base_model.get_config()['layers'])

177

In [46]:
for i in base_model.get_config()['layers']:
    print i
    print ''

{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 3, 224, 224), 'input_dtype': 'float32', 'sparse': False, 'name': 'input_11'}, 'inbound_nodes': [], 'name': 'input_11'}

{'class_name': 'ZeroPadding2D', 'config': {'padding': (3, 3), 'trainable': True, 'name': 'zeropadding2d_11'}, 'inbound_nodes': [[['input_11', 0, 0]]], 'name': 'zeropadding2d_11'}

{'class_name': 'Convolution2D', 'config': {'W_constraint': None, 'b_constraint': None, 'name': 'conv1', 'activity_regularizer': None, 'trainable': True, 'dim_ordering': 'th', 'nb_col': 7, 'subsample': (2, 2), 'init': 'glorot_uniform', 'bias': True, 'nb_filter': 64, 'border_mode': 'valid', 'b_regularizer': None, 'W_regularizer': None, 'activation': 'linear', 'nb_row': 7}, 'inbound_nodes': [[['zeropadding2d_11', 0, 0]]], 'name': 'conv1'}

{'class_name': 'BatchNormalization', 'config': {'gamma_regularizer': None, 'name': 'bn_conv1', 'epsilon': 1e-05, 'trainable': True, 'mode': 0, 'beta_regularizer': None, 'momentum': 0.99, 'axi

In [27]:
for i in base_model.get_config()['layers']:
    print i['name']

input_7
zeropadding2d_7
conv1
bn_conv1
activation_295
maxpooling2d_7
res2a_branch2a
bn2a_branch2a
activation_296
res2a_branch2b
bn2a_branch2b
activation_297
res2a_branch2c
res2a_branch1
bn2a_branch2c
bn2a_branch1
merge_97
activation_298
res2b_branch2a
bn2b_branch2a
activation_299
res2b_branch2b
bn2b_branch2b
activation_300
res2b_branch2c
bn2b_branch2c
merge_98
activation_301
res2c_branch2a
bn2c_branch2a
activation_302
res2c_branch2b
bn2c_branch2b
activation_303
res2c_branch2c
bn2c_branch2c
merge_99
activation_304
res3a_branch2a
bn3a_branch2a
activation_305
res3a_branch2b
bn3a_branch2b
activation_306
res3a_branch2c
res3a_branch1
bn3a_branch2c
bn3a_branch1
merge_100
activation_307
res3b_branch2a
bn3b_branch2a
activation_308
res3b_branch2b
bn3b_branch2b
activation_309
res3b_branch2c
bn3b_branch2c
merge_101
activation_310
res3c_branch2a
bn3c_branch2a
activation_311
res3c_branch2b
bn3c_branch2b
activation_312
res3c_branch2c
bn3c_branch2c
merge_102
activation_313
res3d_branch2a
bn3d_branch2a

In [36]:
fc_feats, time, filename = test()

extracting features for Affenpinscher
file: Affenpinscher3-2.jpeg
took 88.6625988483 seconds


In [47]:
model = create_model('merge_169')
feats, time, filename = test()

extracting features for Affenpinscher
file: Affenpinscher3-2.jpeg
took 29.0210108757 seconds


In [48]:
feats.shape

(1, 1024, 14, 14)